<a href="https://colab.research.google.com/github/alinahasss/grant-writer/blob/main/2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece --quiet
!pip install pytorch-lightning
!pip install torchmetrics
!pip install comet-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
import transformers
import torchmetrics
import pytorch_lightning as pl
import json
import tqdm
import sklearn.metrics
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

pl.utilities.seed.seed_everything(42)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/seed.py:47: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
INFO:lightning_fabric.utilities.seed:Global seed set to 42


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


42

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/hakatonn2023/train.tsv', sep='\t', header=None)
val = pd.read_csv('/content/drive/MyDrive/hakatonn2023/val.tsv', sep='\t', header=None)

In [ ]:
class NlpDataset(torch.utils.data.Dataset):
    def __init__(self,data,labels,tokenizer):
        self.data = data.to_list()
        self.labels = labels.tolist()
        self.encodings = tokenizer(self.data, truncation=True, padding=True)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_name = "cointegrated/rut5-base-multitask"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

### **train**

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
batch_size = 16  # сколько примеров показывем модели за один шаг
report_steps = 200  # раз в сколько шагов печатаем результат
epochs = 3

In [ ]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))